In [2]:
%load_ext autoreload
%autoreload 2
viz_cortex = __import__('03_viz_cortex')
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
import h5py
import pickle as pkl
from copy import deepcopy
import matplotlib.pyplot as plt
import os.path
import sys
sys.path.append('..')
fit_decoding = __import__('02_fit_decoding')

In [4]:
encoding_result='/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03/',
decoding_result = join('/home/chansingh/mntv1/deep-fMRI/results/linear_models/oct25',
                         'coef_rotten_tomatoes_bert-10__ndel=4fmri_perc=0_seed=1.pkl')
args = pkl.load(open(decoding_result.replace('coef_', ''), 'rb')).reset_index()
args = args.iloc[0]
assert args.perc_threshold_fmri == 0, 'Should run this script with perc=0!'
X_train, y_train, X_test, y_test = fit_decoding.data.get_dsets(
        args.dset, seed=args.seed, subsample_frac=0)

feats_train, feats_test = fit_decoding.get_feats(
        args.model, X_train, X_test,
        subject_fmri=args.subject, perc_threshold_fmri=args.perc_threshold_fmri, args=args)

norms = {
        'feats_train_mean': feats_train.mean(axis=0),
        'feats_train_std': feats_train.std(axis=0),
        'feats_test_mean': feats_test.mean(axis=0),
        'feats_test_std': feats_test.std(axis=0),
}
pkl.dump(norms, open)



Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/chansingh/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset rotten_tomatoes (/home/chansingh/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [ ]:
corrs_thresh, coefs = viz_cortex.load_corrs_and_coefs()

In [ ]:
plt.plot(corrs_thresh, coefs, '.')
plt.xlabel('corrs')
plt.ylabel('coefs')
plt.show()

In [3]:

np.zeros((5, 3)).mean(axis=0)

array([0., 0., 0.])